In [1]:
import numpy as np
import gym

## Task 0:

In [2]:
def monte_carlo(env, V, policy, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99):
    """
    Performs the Monte Carlo algorithm to update the value function based on the policy.

    Parameters:
    env: The environment.
    V: The value function.
    policy: The policy.
    episodes: The number of episodes.
    max_steps: The maximum number of steps per episode.
    alpha: The learning rate.
    gamma: The discount factor.

    Returns:
    The updated value function.
    """
    # Creates dictionary with 'n' keys corresponding to a state. Value
    # for each key will be empty list, used later for storing returns from
    # Monte Carlo algorithm.
    returns = {s: [] for s in range(env.observation_space.n)}

    # Resets environment at beginning of each episode and initializes and empty
    # list to store episode-related data
    for episode in range(episodes):
        state = env.reset()
        current_episode = []

        # Executes one episode in environment, following policy to sle
        for step in range(max_steps):
            action = policy(state)
            next_state, reward, done, _ = env.step(action)
            current_episode.append((state, action, reward))
            if done:
                break
            state = next_state

        G = 0
        for state, _, reward in reversed(current_episode):
            G = gamma * G + reward
            returns[state].append(G)
            V[state] = (1 - alpha) * V[state] + alpha * G

    return V


## Task-0 Main File:

In [ ]:
np.random.seed(0)

env = gym.make('FrozenLake8x8-v1')
LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

def policy(s):
    p = np.random.uniform()
    if p > 0.5:
        if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
            return UP
        else:
            return LEFT
    else:
        if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
            return LEFT
        else:
            return UP

V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64') 
np.set_printoptions(precision=2)
env.seed(0)
print(monte_carlo(env, V, policy).reshape((8, 8)))

## Task 1:

In [3]:
def td_lambtha(env, V, policy, lambtha, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99):
    """
    Performs the TD(λ) algorithm to update the value function based on the policy.

    Parameters:
    env: The environment.
    V: The value function.
    policy: The policy.
    lambtha: The trace decay parameter.
    episodes: The number of episodes.
    max_steps: The maximum number of steps per episode.
    alpha: The learning rate.
    gamma: The discount factor.

    Returns:
    The updated value function.
    """
    # Initialize eligibility traces
    E = np.zeros_like(V)

    for episode in range(episodes):
        state = env.reset()

        for step in range(max_steps):
            # Take an action, observe the reward and next state
            action = policy[state]
            next_state, reward, done, _ = env.step(action)

            # Compute the TD error
            td_error = reward + gamma * V[next_state] - V[state]

            # Update eligibility trace
            E[state] += 1

            # Update V and E
            V += alpha * td_error * E
            E *= gamma * lambtha

            if done:
                break

            # Update state
            state = next_state

    return V


## Task-1 Main File:

In [ ]:
np.random.seed(0)

env = gym.make('FrozenLake8x8-v1')
LEFT, DOWN, RIGHT, UP = 0, 1, 2, 3

def policy(s):
    p = np.random.uniform()
    if p > 0.5:
        if s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s // 8 != 0 and env.desc[s // 8 - 1, s % 8] != b'H':
            return UP
        else:
            return LEFT
    else:
        if s // 8 != 7 and env.desc[s // 8 + 1, s % 8] != b'H':
            return DOWN
        elif s % 8 != 7 and env.desc[s // 8, s % 8 + 1] != b'H':
            return RIGHT
        elif s % 8 != 0 and env.desc[s // 8, s % 8 - 1] != b'H':
            return LEFT
        else:
            return UP

V = np.where(env.desc == b'H', -1, 1).reshape(64).astype('float64') 
np.set_printoptions(precision=2)
env.seed(0)
print(monte_carlo(env, V, policy).reshape((8, 8)))


## Task 2:

In [4]:
def sarsa_lambtha(env, Q, lambtha, episodes=5000, max_steps=100, alpha=0.1, gamma=0.99, epsilon=1, min_epsilon=0.1, epsilon_decay=0.05):
    """
    Performs the SARSA(λ) algorithm to update the action-value function based on the policy.

    Parameters:
    env: The environment.
    Q: The action-value function.
    lambtha: The trace decay parameter.
    episodes: The number of episodes.
    max_steps: The maximum number of steps per episode.
    alpha: The learning rate.
    gamma: The discount factor.
    epsilon: The initial epsilon for the epsilon-greedy policy.
    min_epsilon: The minimum epsilon.
    epsilon_decay: The decay rate for epsilon.

    Returns:
    The updated action-value function.
    """
    # Initialize eligibitility traces
    E = np.zeros_like(Q)

    for episode in range(episodes):
        state = env.reset()
        action = np.argmax(Q[state] + np.random.randn(1, env.action_space.n) * (1. / (episode + 1)))

        for step in range(max_steps):
            # Take an action, observe the reward and next state
            next_state, reward, done, _ = env.step(action)

            # Choose next action using epsilon-greedy policy
            next_action = np.argmax(Q[next_state] + np.random.randn(1, env.action_space.n) * (epsilon))

            # Compute the TD error
            td_error = reward + gamma * Q[next_state, next_action] - Q[state, action]

            # Update eligibility trace
            E[state, action] += 1

            # Update Q and E
            Q += alpha * td_error * E
            E *= gamma * lambtha

            # Update state and action
            state = next_state
            action = next_action

        # Decay epsilon
        epsilon = max(min_epsilon, min(epsilon, 1.0 - epsilon_decay * (episode / episodes)))

    return Q


## Task-2 Main File:

In [5]:
np.random.seed(0)
env = gym.make('FrozenLake8x8-v0')
Q = np.random.uniform(size=(64, 4))
np.set_printoptions(precision=4)
print(sarsa_lambtha(env, Q, 0.9))

[[ 3.7642e-03  2.6942e-03  2.5738e-03  2.5432e-03]
 [ 2.3518e-03  4.2909e-03  3.1204e-03  2.3354e-03]
 [ 3.7641e-03  5.2417e-03  2.9398e-03  3.9231e-03]
 [ 3.8798e-03  3.8118e-03  4.1403e-03  1.0694e-02]
 [ 7.1160e-03  1.4607e-02  1.4131e-02  8.1125e-03]
 [ 7.5630e-03  2.7302e-02  1.9421e-02  1.7322e-02]
 [ 4.2409e-02  1.7769e-02  3.1961e-02  1.1431e-02]
 [ 9.4788e-03  1.0871e-02  1.1786e-02  2.5303e-02]
 [ 3.5272e-03  2.8264e-03  4.1816e-03  3.6653e-03]
 [ 2.0944e-03  3.4671e-03  2.7354e-03  6.7579e-03]
 [ 3.1059e-03  9.8110e-03  2.2333e-03  2.7272e-03]
 [ 1.1634e-03  3.2468e-03  5.5223e-03  4.2952e-03]
 [ 1.3937e-02  1.3684e-02  8.0174e-03  6.3906e-03]
 [ 2.2853e-02  1.2878e-02  9.0955e-03  1.1868e-02]
 [ 3.0123e-02  1.6305e-02  2.0007e-02  1.4396e-02]
 [ 1.6091e-02  1.7035e-02  2.0579e-02  1.1247e-02]
 [ 2.3093e-03  1.7324e-03  1.6756e-03  2.0751e-03]
 [ 2.3237e-03  2.6678e-03  2.7045e-03  1.9742e-03]
 [ 1.9408e-03  2.4748e-03  4.4871e-03  2.4119e-03]
 [-5.4027e-04 -5.2404e-04 -5.26